In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsgm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
#Save Kyoto_Gases data (2020-2100), consider the case of C1-C8 for simplicity.
Kyoto_Gases = pd.read_csv('Kyoto Gases.csv')
Kyoto_Gases = Kyoto_Gases[Kyoto_Gases['Category'].isin(['C1','C2','C3','C4','C5','C6','C7','C8'])]
mapping = {'C1':0,'C2':0,'C3':0,'C4':0,'C5':1,'C6':1,'C7':2,'C8':2}#Aggregate categories into 3 categories, with 0-2 corresponding to C123-C78, respectively
Kyoto_Gases['Category'].replace(mapping,inplace=True)
Kyoto_Gases.reset_index(drop=True,inplace=True)

In [3]:
Kyoto_Gases.drop(columns=['Category_name'],inplace = True)

In [4]:
#Load a dataset of individual variables
CarbonSequestration = pd.read_csv('Carbon_Sequestration_CCS_imputed.csv')
FinalEnergy_Liquid = pd.read_csv('Final Energy_Liquids.csv')
PrimaryEnergy_Gas = pd.read_csv('Primary Energy_Gas.csv')
PrimaryEnergy_Oil = pd.read_csv('Primary Energy_Oil.csv')
PrimaryEnergy_Coal = pd.read_csv('PrimaryEnergy_Coal.csv')

In [5]:
#Get the intersection of the models and scenarios contained in each variable
Model_Scenario = Kyoto_Gases[['Model','Scenario']]
Variables = [CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil]
for variable in Variables:
    Model_Scenario = pd.merge(Model_Scenario,variable[['Model','Scenario']],on=['Model','Scenario'],how='inner')

In [6]:
for i in range(len(Variables)):
    Variables[i] = pd.merge(Model_Scenario,Variables[i],on=['Model','Scenario'],how='inner')
for i in range(len(Variables)):
    Variables[i].drop(columns=['Category_name'],inplace = True)

In [7]:
Kyoto_Gases = pd.merge(Kyoto_Gases,Model_Scenario,on = ['Model','Scenario'],how = 'inner')

In [8]:
Variables.append(Kyoto_Gases)
#Variables :[CarbonSequestration,FinalEnergy_Liquid,PrimaryEnergy_Coal,PrimaryEnergy_Gas,PrimaryEnergy_Oil,Kyoto_Gases]

In [9]:
#Generate feature matrices, the values of each variable during 2020-2100. 9 time steps, 6 features
#1160 is the amount of data
X = np.zeros((1160,9,6))
for i in range(len(Variables)):
    Variables[i] = Variables[i].iloc[:,3:-1].values
for i in range(1160):
    for j in range(9):
        for k in range(6):
            X[i][j][k] = (Variables[k])[i,j]

In [10]:
Y = Kyoto_Gases['Category'].values

In [11]:
#Separate datasets by category
C1234_DataSet = X[Y == 0]
C56_DataSet = X[Y == 1]
C78_DataSet = X[Y == 2]

In [12]:
#Define the model structure for C1234
architecture1 = tsgm.models.zoo["vae_conv5"](9, 6, 8)#Latent Dim = 8
encoder1, decoder1 = architecture1.encoder, architecture1.decoder
encoder1.load_weights('encoder1_weights.h5')
decoder1.load_weights('decoder1_weights.h5')

In [13]:
#Standardize the data and compress it to (0, 1).
scaler_C1234 = tsgm.utils.TSFeatureWiseScaler((0,1))        
scaled_C1234_data = scaler_C1234.fit_transform(C1234_DataSet)

In [ ]:
#Define an optimization strategy
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=200,
    decay_rate=0.98)
vae_C1234 = tsgm.models.cvae.BetaVAE(encoder1, decoder1)
vae_C1234.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C1234.fit(scaled_C1234_data, epochs=2000, batch_size=50)

In [17]:
#Define the model structure for C56
architecture2 = tsgm.models.zoo["vae_conv5"](9, 6, 8)
encoder2, decoder2 = architecture2.encoder, architecture2.decoder
scaler_C56 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C56_data = scaler_C56.fit_transform(C56_DataSet)
encoder2.load_weights('encoder2_weights.h5')
decoder2.load_weights('decoder2_weights.h5')

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=200,
    decay_rate=0.98)
vae_C56 = tsgm.models.cvae.BetaVAE(encoder2, decoder2)
vae_C56.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C56.fit(scaled_C56_data, epochs=2000, batch_size=30)

In [20]:
#Define the model structure for C78
architecture3 = tsgm.models.zoo["vae_conv5"](9, 6, 8)
encoder3, decoder3 = architecture3.encoder, architecture3.decoder
scaler_C78 = tsgm.utils.TSFeatureWiseScaler((0,1))    
scaled_C78_data = scaler_C78.fit_transform(C78_DataSet)
encoder3.load_weights('encoder3_weights.h5')
decoder3.load_weights('decoder3_weights.h5')

In [ ]:
vae_C78 = tsgm.models.cvae.BetaVAE(encoder3, decoder3)
vae_C78.compile(optimizer=keras.optimizers.Adam(lr_schedule))
vae_C78.fit(scaled_C78_data, epochs=2000, batch_size=30)

In [22]:
#Set Global Random Seed
global_seed = 8
tf.random.set_seed(global_seed)
np.random.seed(global_seed)

In [24]:
#Generate data using generative models (500 for each class)
z1 = tf.random.normal((500, 8))
z2 = tf.random.normal((500, 8))
z3 = tf.random.normal((500, 8))
Gen_C1234 = decoder1(z1)
Gen_C56 = decoder2(z2)
Gen_C78 = decoder3(z3)
Gen_C1234 = scaler_C1234.inverse_transform(Gen_C1234)
Gen_C56 = scaler_C56.inverse_transform(Gen_C56)
Gen_C78 = scaler_C78.inverse_transform(Gen_C78)

In [ ]:
#CarbonSequestration_2050
import matplotlib.pyplot as plt
import seaborn as sns
Category_names = ['C1234','C56','C78']
Data_Values = [Gen_C1234[:,3,0],Gen_C56[:,3,0],Gen_C78[:,3,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for CarbonSequestration_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,30000])
#plt.savefig('Box Plot of Agg-Categories for CarbonSequestration_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,2],Gen_C56[:,3,2],Gen_C78[:,3,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,380])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,3],Gen_C56[:,3,3],Gen_C78[:,3,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,350])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,4],Gen_C56[:,3,4],Gen_C78[:,3,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,350])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,1],Gen_C56[:,3,1],Gen_C78[:,3,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for FinalEnergy_Liquid_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,350])
#plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
Data_Values = [Gen_C1234[:,3,5],Gen_C56[:,3,5],Gen_C78[:,3,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25
colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2050',fontsize=20)
plt.title('Box Plot of Agg-Categories for Kyoto_Gases_2050_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,350])
#plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2050_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Coal_2100
Data_Values = [Gen_C1234[:,8,2],Gen_C56[:,8,2],Gen_C78[:,8,2]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Coal_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,780])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Coal_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Gas_2100
Data_Values = [Gen_C1234[:,8,3],Gen_C56[:,8,3],Gen_C78[:,8,3]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Gas_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,380])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Gas_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#PrimaryEnergy_Oil_2100
Data_Values = [Gen_C1234[:,8,4],Gen_C56[:,8,4],Gen_C78[:,8,4]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('PrimaryEnergy_Oil_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,300])
#plt.savefig('Box Plot of Agg-Categories for PrimaryEnergy_Oil_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#CarbonSequestration_2100
Data_Values = [Gen_C1234[:,8,0],Gen_C56[:,8,0],Gen_C78[:,8,0]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('CarbonSequestration_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for CarbonSequestration_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,40000])
#plt.savefig('Box Plot of Agg-Categories for CarbonSequestration_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#FinalEnergy_Liquid_2100
Data_Values = [Gen_C1234[:,8,1],Gen_C56[:,8,1],Gen_C78[:,8,1]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('FinalEnergy_Liquid_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for FinalEnergy_Liquid_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,380])
#plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2100_Generate.pdf',format='pdf')
plt.show()

In [ ]:
#Kyoto_Gases_2100
Data_Values = [Gen_C1234[:,8,5],Gen_C56[:,8,5],Gen_C78[:,8,5]]
box_positions = np.arange(1, len(Category_names) + 1) - 0.25
scatter_positions = np.arange(1, len(Category_names) + 1) + 0.25

colors = ['goldenrod','royalblue','brown','orangered','olive','green','darkcyan','blueviolet']
plt.figure(figsize=(12,10))
box_plot = plt.boxplot(Data_Values,showfliers=False,patch_artist=True,widths=0.15,positions=box_positions)
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
for median_line in box_plot['medians']:
    median_line.set(color='black')
# scatterplot
for i, data in enumerate(Data_Values):
    x = np.random.normal(scatter_positions[i], 0.05, size=len(data))
    plt.scatter(x, data, color=colors[i], s=5.8)
plt.xlabel('Agg-Category',fontsize=20)
plt.ylabel('Kyoto_Gases_2100',fontsize=20)
plt.title('Box Plot of Agg-Categories for Kyoto_Gases_2100_Generate',fontsize=25)
# set scale
plt.xticks(range(1, len(Category_names) + 1), Category_names)
#plt.ylim([0,380])
#plt.savefig('Box Plot of Agg-Categories for FinalEnergy_Liquid_2100_Generate.pdf',format='pdf')
plt.show()

In [38]:
encoder1.save_weights('Policy-encoder1_weights-top6.h5')
encoder2.save_weights('Policy-encoder2_weights-top6.h5')
encoder3.save_weights('Policy-encoder3_weights-top6.h5')
decoder1.save_weights('Policy-decoder1_weights-top6.h5')
decoder2.save_weights('Policy-decoder2_weights-top6.h5')
decoder3.save_weights('Policy-decoder3_weights-top6.h5')